## 本地读取数据转化成 df

In [ ]:
case class AppLogRawStruct(
      server_ip: String,
      createtime: String,
      server_name: String,
      func_name: String,
      imei: String,
      userid: String,
      account_id: String,
      app_version: String,
      os_version: String,
      platform: String,
      channel: String,
      token: String,
      return_code: String,
      invoke_ret_code: String,
      clientip: String,
      request: String,
      reverse: String,
      reverse1: String,
      response: String
) extends Serializable {}


In [ ]:
case class AppLogRawStruct(
      server_ip: Option[String],
      createtime: Option[String],
      server_name: Option[String],
      func_name: Option[String],
      imei: Option[String],
      userid: Option[String],
      account_id: Option[String],
      app_version: Option[String],
      os_version: Option[String],
      platform: Option[String],
      channel: Option[String],
      token: Option[String],
      return_code: Option[String],
      invoke_ret_code: Option[String],
      clientip: Option[String],
      request: Option[String],
      reverse: Option[String],
      reverse1: Option[String],
      response: Option[String]
) extends Serializable {}


In [0]:
import org.json4s.{JArray, JNothing, JObject, JValue}

case class ResLimitStatus(
    status: Option[Int]
) {}

case class ResRoute(
    adsorb_info: Option[String],
    args: Option[String],
    bounds: Option[String],
    data_version: Option[Long],
    coors: Option[String],
    limit_status: Option[ResLimitStatus]
    

) {}

case class ResDetail(
   mt: Option[List[ResRoute]],
   limit_status: Option[ResLimitStatus]
) {}

// case class Response(
//     detail: Option[JValue],
//     data_version: Option[Long],
//     coors: Option[String],
//     limit_status: Option[JValue],
//     args: Option[String],
//     bounds: Option[String]
// ) {}

case class Response(
    detail: Option[JValue],
    data_version: Option[JValue],
    coors: Option[JValue],
    limit_status: Option[JValue],
    args: Option[JValue],
    bounds: Option[JValue]
) {}

cmd0.sc:1: object json4s is not a member of package org
import org.json4s.{JArray, JNothing, JObject, JValue}
           ^cmd0.sc:33: not found: type JValue
    detail: Option[JValue],
                   ^cmd0.sc:34: not found: type JValue
    data_version: Option[JValue],
                         ^cmd0.sc:35: not found: type JValue
    coors: Option[JValue],
                  ^cmd0.sc:36: not found: type JValue
    limit_status: Option[JValue],
                         ^cmd0.sc:37: not found: type JValue
    args: Option[JValue],
                 ^cmd0.sc:38: not found: type JValue
    bounds: Option[JValue]
                   ^Compilation Failed

: 

In [ ]:
import org.apache.spark.sql.{Row, Dataset, SparkSession}
import org.apache.spark.sql.{Encoder, Encoders, SaveMode}
import org.json4s.jackson.JsonMethods.{compact, parse, render}
import org.json4s._
import org.json4s.jackson.JsonMethods
import org.json4s.jackson.Serialization.{read}
import org.json4s.JsonDSL._

val spark = SparkSession
      .builder()
      .appName("test")
      .master("local")
      .config("hive.exec.dynamic.partition", "true")
      .config("hive.exec.dynamic.partition.mode", "nonstrict")
      .config("hive.exec.compress.output", "true")
      .config("spark.driver.memory", "2g")
      .config("spark.executor.memory", "2g")
      .getOrCreate()
    // For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

// val fileName = "/Users/andyzhuang/tmp/session_log/logpoolserver_2020010710"
val fileName = "/Users/andyzhuang/tmp/session_log/logpool_2020020810"


val localFileName = "file://"+fileName
val logSchema = Encoders.product[AppLogRawStruct].schema
val df = spark.read
            .format("csv")
            .option("header", "false")
            .option("delimiter", "|")
            .schema(logSchema)
            .load(localFileName)
// df.show(2)

val ds:Dataset[AppLogRawStruct] = df.as[AppLogRawStruct]
val rdd = ds.rdd
.filter(x=>{
   x.response.isDefined 
})
.map(_.response).filter(_.isDefined).map(_.get)

rdd.toDF.show(1)
val x = rdd.toDF.take(1)
val v = x(0).getAs[String](0)
// println(v)
val json = parse(v)

implicit val formats = DefaultFormats + FieldSerializer[Response]()

val obj = read[Response](v)
// println(json)
// val obj = json.extract[Response]
println(obj)

spark.close

## 为什么这个的环境和运行的环境不一样，弄得我这个乱七八糟的，浪费了一天时间

In [0]:
%AddDeps "graphframes" % "graphframes" % "0.1.0-spark1.6" --repository http://dl.bintray.com/spark-packages/maven

(console):1: identifier expected but eof found.
%AddDeps "graphframes" % "graphframes" % "0.1.0-spark1.6" --repository http://dl.bintray.com/spark-packages/maven
                                                                                                                 ^

: 

In [ ]:
import org.json4s.jackson.JsonMethods.{compact, parse, render}
import org.json4s._
import org.json4s.jackson.JsonMethods
import org.json4s.jackson.Serialization.{read}
import org.json4s.JsonDSL._

val v= """
{"detail":{"mt":[{"adsorb_info":"1378988445000505254;SR_INHERIT;261400;ST_NONE;ADSORB_POI_PART_INVALID,;;END_NPT_ONLINE&throwedge_2;adsorb_type=gpsmatcher_normal;region_uid_0;region_no_judge;region_status_0;region_scale_0;bound_type_0;model_type_10","args":"&cond=0&nohighway=0&notoll=0&traffic=1&noround=0","bounds":"12754038.81,4086119.52,12834742.02,4136688.95","close_info":{"closed_pos_info":0,"dest_isolate_info":0,"segments":[],"start_isolate_info":0},"color":"#0079ff","coors":"12754110.43,4086135.04,-7162,472,0,0,10241,-675,11355,-1349,13472,195,7901,749,5121,1485,16698,7288,9573,4858,27665,12276,16752,7698,2449,1350,32505,14575,8572,4049,17412,7636,6698,2173,25538,5442,24713,6343,34905,7087,7194,898,29916,6231,51161,14999,33117,10871,23154,6478,8015,1890,28053,8368,60001,17546,10241,2430,51541,15522,47088,13498,34955,9718,49982,14848,113435,32261,48090,4049,22704,1233,42306,2682,0,0,4230,4724,9017,9449,24936,25243,4564,4320,4898,5265,6790,6884,10576,10260,25047,25244,6234,7019,15995,16544,30091,30301,3339,4455,2561,5941,2942,8605,1177,4220,2226,9451,3228,10260,1341,5215,935,4627,2333,7382,5410,13423,2226,4996,1008,2978,358,1970,982,20720,1166,16258,2720,18831,3643,13681,831,2288,3541,12385,2138,4465,1991,5613,6698,16473,15333,29620,4342,6617,7101,11566,5122,7723,5357,9533,8353,13712,2899,7968,7344,12754,2916,5753,5358,9534,1546,3395,624,2652,1691,27362,2601,35617,275,19935,623,11436,1611,12188,7805,13731,4790,6165,7477,10153,2943,3034,3046,1987,13037,5641,3892,2082,5446,3757,1893,1756,2449,3646,12245,24446,4786,10805,3340,6213,4008,9590,8794,17829,8794,15668,6902,10265,10130,16884,5789,10130,4564,7024,6714,12396,4306,6650,4342,5268,9796,9726,15919,9590,8460,4323,21173,13278,4096,2391,49535,30582,19483,11700,23711,13509,31838,16211,3451,1486,50427,25532,18573,10321,17940,9538,15808,10133,24267,18508,7904,6755,24045,17834,31058,24048,11021,9188,8184,5678,22095,17695,32950,24861,36513,28104,10019,7296,4564,3783,2783,2703,24601,18646,53434,41888,15028,12161,10687,7703,13247,11080,15028,10811,21373,16756,5789,3919,4898,3783,0,0,1447,1217,-1113,1756,-11021,16487,-4119,5540,-3673,4190,-1225,1621,-10798,10406,-8349,7703,-3228,3243,-2783,3243,-12356,16623,-7681,8919,-6012,5811,-12579,11352,-3673,3514,-1225,1622,-1447,2568,-557,2162,-222,19326,-223,2433,-328,992,0,0,-1339,1501,-1747,522,-731,59,-25053,482,-9764,-178,-17032,271,-6790,-135,-57441,811,0,0,-2338,405,-1558,541,-1447,1216,-780,1351,-111,61630,223,9867,334,39737,334,50552,1558,219536,334,15413,-111,6624,644,41192,-199,8022,334,7301,6123,50432,7903,67202,2004,18255,9471,81243,1772,15715,-222,15687,-334,13524,0,0,-668,11360,-1225,7979,-1224,5003,-2561,7709,0,0,-667,2705,-2561,7438,-890,4193,-223,2164,0,2299,223,2164,556,2164,779,2163,1002,2029,2561,3652,1558,1623,6345,4598,25159,8656,25937,7979,18702,4463,13692,2299,14249,2029,16141,1488,16921,676,13803,-135,16587,-676,16030,-1623,13024,-1894,10019,-1758,13692,-2975,30391,-8250,13024,-3923,23822,-6086,24602,-5545,21930,-4057,20483,-3111,28386,-2975,17255,-1488,16141,-1082,31503,-2704,49649,-3652,11911,-1217,24379,-3381,23934,-3922,13692,-2570,9685,-2164,18590,-4462,16364,-4869,114103,-35703,18479,-5274,9796,-2299,18479,-3381,10353,-1217,9350,-811,9240,-541,11688,-271,10687,271,19258,1623,12246,1758,14137,2434,10464,2705,16475,4462,18813,6356,26161,8385,37737,12442,7792,2705,61449,20015,20371,5951,15251,4057,21150,5004,25827,5545,25492,4598,41856,4869,25492,1758,22598,947,31149,2184,28852,3090,38739,5951,8795,1488,87163,17176,19238,3237,7144,821,15808,1217,15139,812,15585,135,9240,-271,14582,-946,19036,-2164,10910,-1830,4978,-537,21180,-4977,73163,-14470,29221,-4323,7972,-882,19288,-1671,24842,-1195,34636,-1015,11922,469,20817,541,16475,1217,5232,541,30613,2435,18034,1217,37403,2164,15140,676,17811,406,25937,135,11002,-35,25845,-912,20817,-1487,23043,-1894,24045,-2299,20594,-1623,17143,-811,18702,-271,21930,135,16587,541,14582,677,56439,5680,43860,4733,33507,2435,29945,541,8127,0,6456,-406,23155,-541,61337,-3111,52543,-2434,22375,-406,38405,947,8126,406,39407,3246,29945,2840,23266,1623,30724,676,24157,-406,24267,-1217,15585,-1217,16253,-2029,32060,-3381,75808,-8520,33062,-3382,22821,-2975,11911,-1217,23377,-2705,21039,-1893,20594,-1623,52766,-1759,25269,-135,26940,1353,29722,1758,28609,2705,22264,2705,29834,4327,16698,2841,47310,9737,73582,16906,18034,4463,25381,5951,14360,3652,17923,5410,38850,12849,5121,1758,14917,5546,12245,4734,46643,19882,6679,2976,22709,9468,12913,5816,38739,16501,11689,5140,11243,4599,42413,18531,20483,8657,13358,3923,3896,811,7681,541,6012,-135,5565,-676,7459,-2029,5343,-2029,5900,-2976,4119,-2705,5789,-4599,19703,-19884,3896,-2435,45530,-50315,45084,-49503,25159,-27320,18701,-21098,28943,-31917,24491,-27318,4341,-4462,16475,-18122,26383,-27046,20372,-18662,17922,-15821,19370,-15957,19814,-15415,4453,-3110,16030,-11764,11763,-8226,0,0,3599,-3133,4564,-3245,4898,-3786,2450,-2705,779,-1217,556,-2163,112,-1623,-223,-1487,-557,-1623,-556,-946,-1113,-1217,-2004,-1217,-2783,-541,-1225,135,-1669,541,-1114,676,-1335,1488,-446,676,-890,2028,-334,2299,0,1217,779,5003,2004,7707,1558,6761,779,5409,-111,3921,-334,2975,-1113,3786,-557,1217,-2337,3787,-3117,4191,-5900,7438,0,0,-6457,7031,-6234,5680,0,0,1002,3110,5455,14199,2437,5685,4131,7297,7124,11494,14472,19473,6345,9602,1781,3110,2912,5695,4435,8099,13358,27589,11355,22991,11689,22585,6901,11496,14027,24074,5120,9197,0,0,-556,2029,-11466,811,-11473,471,-19861,1580,0,0,-47,6631,1213,26754,5455,108345,0,2434,3213,59605,349,10602,4600,103914,-136,3711,16242,136440,13024,103919,557,5278,0,1218,19258,59947,23370,73823,7707,23914,4477,13250,0,0,1871,7727,8438,29479,6484,18298,4902,16236,3791,9900,3171,5376,3507,4412,5397,4571,7722,3821,8510,2010,18076,2238,19147,1082,20640,457,24429,989,17155,1352,19724,1013,34675,2079,78186,4127,0,0,8069,17678,2195,5693,481,1990,-279,9222,-692,6209,-2767,17499,-2066,15319,-1195,6898,-1015,9883,-123,4089,507,12065,377,14594,1295,24240,111,13131,2672,50630,1802,30847,4432,86533,5343,54294,2844,46676,-895,3612,-5622,13622,-16547,36415,-17072,38739,-1892,6635,-668,4198,222,4604,20904,109138,1583,9224,3562,34536,557,14627,111,4063,-890,27901,-392,23731,392,12704,-112,19098,-1113,27632,-2004,75314,-668,72474,264,28925,-598,9414,-3117,24657,-890,8670,-668,36038,1336,40646,-203,14468,362,14165,538,13464,761,5546,518,11883,394,20881,0,0,23679,-4666,5220,-870,7915,-1840,9908,-2033,7124,-948,8238,-813,10909,135,67571,-3387,10798,-136,59795,-1895,0,0,10597,6168,7546,4972,19516,12211,6790,3979,3258,1715,2504,651,5321,518,22041,813,5010,949,11911,4607,8794,2845,16587,5014,37069,13550,3896,678,43860,-1220,5789,542,5454,813,3451,1626,891,678,7570,6911,15696,16531,7013,6640,20473,21149,12700,12864,21636,21361,16146,16500,9892,10881,14309,18338,9818,11384,5575,7118,13572,16327,11800,14772,14418,16872,8736,9962,5232,6912,7384,8709,4356,5901,18539,22119,24491,33207,5788,8674,5789,7184,5822,8192,8433,10560,5872,6984,0,0,40097,-3779,6862,-489,39517,-4138,27203,-2556,10796,-1174,0,0,2006,15270,3896,41884,779,4202,1113,11250,4008,34702,779,7998,993,13165,0,0,6577,-559,13024,-2169,12134,-813,77923,-8404,27385,-3254,49760,-5151,29945,-3524,23934,-3118,19258,-2169,6122,-271,16142,-2440,15584,-1627,21374,-2846,9462,-542,33287,-3529,20703,-2842,7907,-883,4513,-204,47581,-5691,17923,-3118,10405,-1580,13751,-3028,9462,-1356,22573,-4091,2585,-653,5473,-968,56038,-8821,0,0,3722,32155,1670,15724,-38,6027,597,6658,3449,23237,668,9625,0,9625,-223,7320,111,3796,1002,4473,557,7456,556,12472,0,0,90984,-12721,20002,-1920,20797,-2841,36074,-4563,25549,-3911,0,0,1322,-202,1966,16962","cost":0,"data_version":1580995505,"dest_region_coors":"","distance":106788,"endInfo":{"dir":"","distance":0},"end_adsorb_type":3,"evaluate_score":0.000000,"fee":0,"forbidden_info":{"city_list":"","only_by_passport":0,"reminder":""},"g_label":"CBR.Y.8","high_risk_info":{"dest_in_risk":0},"jam_index":0,"key_roads":["郑民高速","商丘市"],"label":"","link_node":"12754141.22,4086133.01,-10241,675,10241,-675,11355,-1349,13472,195,9147,1111,40388,17858,17423,7687,23431,10937,14805,6613,11037,4976,8110,3797,45864,15250,6679,1350,24713,6343,4629,894,30276,6193,7194,898,29916,6231,51161,14999,12128,4074,20989,6797,2735,680,20419,5798,36068,10258,32171,9448,4830,1405,4222,1230,21115,6003,17923,4859,32352,9781,25595,7446,5394,1535,20594,5939,33952,9448,12579,3645,17811,5264,25270,7559,35358,10045,17741,5073,11132,3104,26523,7568,33608,7316,2097,235,24602,2024,3515,289,3052,251,26601,1638,31869,2054,10437,628,4230,4724,3551,3735,5911,6119,36513,36987,12035,12030,95285,123552,2602,10880,1706,5472,6037,15333,4817,34169,1673,18109,2107,13991,3506,13165,8791,25463,38306,71287,16609,31213,7344,12754,14027,73517,3218,54355,39098,40711,8187,5079,4825,5217,6701,12848,10998,23348,4342,8644,5324,11861,6476,13127,6568,11751,2449,4187,9796,15128,7013,11751,2449,4187,1224,2161,8349,13913,2449,4458,112,270,2484,4561,4752,7190,20928,18776,46309,27692,45638,28150,43194,25209,31838,16211,3451,1486,50427,25532,18573,10321,15491,8322,23266,15266,29388,22967,2284,1701,8960,6676,1668,1228,8907,6608,18702,14456,11479,8913,20768,16154,15392,12294,2788,2225,32128,24485,59080,45716,2523,1950,39741,30537,8127,6351,33388,26352,8134,6618,1681,1210,4330,3114,17923,14459,6989,5028,19393,14701,557,406,20149,15134,1447,1217,-1113,1756,-20372,28108,-668,676,-31273,33144,-38301,42265,-223,3649,-662,19913,-1339,1501,-27531,1063,-9764,-178,-22375,136,-1447,0,-57441,811,-3896,946,-2227,2567,-23,32752,-88,8335,0,10327,0,10216,223,9867,0,1081,0,9867,156,9764,155,9802,23,1384,0,7839,53,8046,207,31332,74,11174,0,8921,0,1217,111,8651,223,31225,673,91251,154,27474,397,50797,223,14061,111,1352,334,32042,223,27852,4841,42360,1393,11317,5232,44486,4675,40971,9471,81243,1550,31402,-334,13524,-1670,17986,-4008,14065,8683,37192,13136,4598,30613,9873,119668,12037,776009,-94940,33952,11225,54624,18037,33063,10133,5710,1718,87720,19204,9573,1082,390843,39222,139452,-26137,98660,-4294,506150,12064,29408,2137,44528,135,23155,-541,24379,-1217,525762,-18123,797604,171097,2560,812,78814,-35169,5010,-5410,87052,-95895,18367,-20017,217630,-212857,5009,-3651,11763,-8226,8609,-6784,-5232,31507,-9017,11629,-3785,4192,-4342,4327,-4564,4192,1002,3110,2665,6868,5227,13016,3463,6080,3734,6261,607,906,1114,1893,1447,2299,890,1352,14866,20217,10644,17663,20688,41503,4622,9292,6539,13103,10880,20477,6011,10143,4564,7845,4676,7979,3228,5545,5677,10144,1225,1894,-1781,135,-26734,1594,-16066,1268,-47,6631,768,16340,1100,23530,1902,38143,1122,22198,1776,34888,0,1623,0,811,2115,40040,445,7711,653,11854,0,1537,776,18803,1243,28410,1224,28274,252,5397,1079,23108,239,12698,1213,10342,1511,12728,1581,13315,1807,15221,2346,19615,5001,41811,1545,12997,1238,10411,2357,18887,973,7793,549,4398,2204,16738,2343,19691,734,5877,1636,13108,892,7143,1336,10284,557,5278,0,1218,6838,21310,1556,4848,5632,17550,5232,16239,13136,41546,5009,15834,5225,16443,7707,23914,4477,13250,1871,7727,2820,9769,4290,14857,661,2281,667,2572,6484,18298,752,2507,661,2203,1661,5531,1828,5995,32098,30090,19263,2305,38600,1472,24429,989,17155,1352,25080,1333,39306,2293,11367,609,7078,379,49754,2605,3805,8420,4264,9258,2676,7683,-279,9222,-1940,14379,-1859,11751,-814,6381,-912,6516,-1826,32935,1672,41947,4585,91495,679,13024,1081,21790,674,13413,996,19757,779,14487,2895,31005,3416,38946,1218,21409,41,711,334,4739,506,8222,-6517,17234,-2338,5281,-14209,31134,-13955,31697,-2894,36292,2449,12865,15894,82460,581,3671,2449,19638,1477,13651,972,15874,223,6637,-321,18833,-850,35778,339,17491,-59,5507,-1113,36436,-1222,42678,-670,28301,-223,14766,-217,20322,-190,19805,-38,5931,0,3793,-112,12192,223,16799,0,7574,-301,10288,-2259,19258,-1893,16664,-492,23230,-287,13891,223,6097,1113,34549,445,37293,1013,10350,109,2515,409,9368,394,20881,13964,-2702,5930,-1151,3785,-813,5220,-870,7915,-1840,16253,-2846,25826,-1084,1447,-135,59345,-2938,11677,-179,37150,-1218,5776,-143,16869,-534,64104,30485,30390,6098,37669,12301,10931,3951,9430,3501,51035,1155,16726,3618,13090,12724,17189,17358,20473,21149,34336,34225,16146,16500,9892,10881,7301,9376,7766,9828,9060,10518,3451,4472,15696,18973,7904,9893,22598,26699,21424,26536,16403,19571,6366,8241,2007,2660,5125,7104,7698,10516,14336,19839,5308,7087,11619,14918,5872,6984,9514,-935,30583,-2844,34509,-3388,35797,-3504,3276,-291,8096,-853,2700,-321,1226,8900,2196,21433,1144,12182,2115,18841,445,4337,3289,29620,1387,11995,1336,15182,436,5981,3825,-254,7093,-1118,40854,-4338,19593,-2033,38293,-4202,35153,-3990,23541,-2528,18451,-1887,35177,-4202,44082,-4880,6457,-949,62810,-7280,26582,-2755,20703,-2842,12420,-1087,6500,-803,34847,-4210,24157,-3796,10405,-1580,13751,-3028,1340,-210,2222,-333,28473,-4904,38452,-6479,25644,-3963,3722,32155,1670,15724,2561,25756,1447,10166,445,26570,2226,28197,90984,-12721,1626,-189,18376,-1731,20797,-2841,36074,-4563,25549,-3911,1322,-202,2057,17703","model_name":"","price":0,"rank_debug_msg":{},"reasons":[],"recommend":"","render_segments":[],"route_diff":-1.000000,"route_limit_info":{"limit_coor_idxs":[],"violated_rule_ids":[]},"route_tips":{"background_color":"#D9F84F4F","closed_road_name":"","closed_road_position_mask":0,"display_text":"","duration":0,"h5_url":"","show_close_button":1,"show_count_down":0,"text_color":"#FFFFFF"},"routeid":7082286592709298092,"score_explain":"","segmentLayer":[{"control":[],"exitName":"","grade":2,"name":"开封市","roadLength":30884,"segmentEnd":7,"segmentNum":7,"segmentStart":0},{"control":[],"exitName":"民权\/睢县\/S211","grade":0,"name":"郑民高速","roadLength":36549,"segmentEnd":8,"segmentNum":1,"segmentStart":7},{"control":[],"exitName":"","grade":0,"name":"商丘市","roadLength":39364,"segmentEnd":23,"segmentNum":15,"segmentStart":8}],"segmentList":[{"accessorialInfo":"","action":"左转调头","action_length":0,"adcode":410222,"alias":"","br":[],"buildingLength":0,"city_borders":[{"adcode":410222,"city_name":"开封市","coor_start":1,"pointx":"12754038.81","pointy":"4086139.76","province_name":"河南省"},{"bound_coors":"12754050.35,4086126.96,126,2434,-2435,127,-126,-2434","coor_start_a":1,"coor_start_b":2,"coor_start_c":8,"pointx_a":"114571479","pointx_b":"114571479","pointx_c":"114577299","pointy_a":"34425939","pointy_b":"34425939","pointy_c":"34426509"}],"connect_len":0,"coorNum":2,"coorStart":0,"directText":"","direction":"西","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":2,"inter":[],"intersection":4,"intersection_ext":0,"intersection_infos":[],"intersections":[4],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":59,"roadName":"S325","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":0,"pointx":"12754110","pointy":"4086134","seg_speed":70,"src":"1"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":2,"textInfo":"沿S325向西行驶59米,左转调头","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":0,"adcode":410222,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12763350.84,4088597.86,212,-2428,2429,212,-212,2428","coor_start_a":36,"coor_start_b":37,"coor_start_c":41,"pointx_a":"114655249","pointx_b":"114655249","pointx_c":"114659089","pointy_a":"34444069","pointy_b":"34444069","pointy_c":"34447309"}],"connect_len":0,"coorNum":35,"coorStart":2,"directText":"","direction":"","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":2,"inter":[{"coorStart":35,"direction":1,"dist":349,"pointx":"12762940","pointy":"4088559","start_distance":349,"tsection":""}],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[],"light":[{"coorStart":3,"name":"红绿灯","pointx":"12754141","pointy":"4086132","start_distance":0,"tips_type":0},{"coorStart":10,"name":"红绿灯","pointx":"12755059","pointy":"4086387","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":86,"roadLength":8003,"roadName":"S325","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":2,"pointx":"12754038","pointy":"4086139","seg_speed":70,"src":"1"},{"coorStart":28,"pointx":"12759363","pointy":"4087678","seg_speed":80,"src":"3"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":2,"textInfo":"沿S325行驶8.0公里,左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":3,"fb_sign":0,"grade":2,"high_risk":0,"len":0,"light":1,"name":"S325","next_sapa_dist":0,"pointx":"12754141","pointy":"4086132","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":9,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12754929","pointy":"4086306","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"2373137905867342957","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":9,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12754939","pointy":"4086335","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2510441","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":9,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12754919","pointy":"4086327","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":32,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":10,"fb_sign":0,"grade":2,"high_risk":0,"len":0,"light":1,"name":"S325","next_sapa_dist":0,"pointx":"12755059","pointy":"4086387","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"cross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":16,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12756111","pointy":"4086858","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"12981830486366709996","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":19,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12756804","pointy":"4086957","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2510514","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":29,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"陈庄诚信石化加油站","next_sapa_dist":0,"pointx":"12759888","pointy":"4087860","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"18070260618896633898","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":30,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"方正石油加油站","next_sapa_dist":0,"pointx":"12760271","pointy":"4087917","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"8937625386544453292","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":31,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站(赵辉)","next_sapa_dist":0,"pointx":"12760901","pointy":"4088164","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"18392732143447007943","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":31,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"中东石化加油站","next_sapa_dist":0,"pointx":"12761045","pointy":"4088193","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"3998644706513652733","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":31,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12761030","pointy":"4088164","section_id":0,"section_length":0,"speed":80,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2510571","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":32,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12761648","pointy":"4088407","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"7263407184132562310","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":32,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12761602","pointy":"4088327","section_id":0,"section_length":0,"speed":80,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2510580","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":33,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12762645","pointy":"4088484","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"10826166941722750485","url1":"","url2":"","view_url":""}],"tsection":"cross","voice_flag":0,"white_bounds":[{"coors":"12754901.33,4086318.22,3914,1692,1692,-3914,-3913,-1692","type":6},{"coors":"12756170.75,4086819.74,-9853,-1981,-1981,9852,9852,1982","type":6}]},{"accessorialInfo":"","action":"左转","action_length":15,"adcode":410221,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12774122.33,4100897.71,-2666,3292,3372,2730,2666,-3292","coor_start_a":145,"coor_start_b":147,"coor_start_c":154,"pointx_a":"114751859","pointx_b":"114751989","pointx_c":"114748369","pointy_a":"34535399","pointy_b":"34535489","pointy_c":"34539019"}],"connect_len":0,"coorNum":109,"coorStart":37,"directText":"","direction":"","driveTime":0,"end_light":1,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[],"light":[{"coorStart":145,"name":"红绿灯","pointx":"12774118","pointy":"4100921","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":14168,"roadName":"X010","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":37,"pointx":"12763363","pointy":"4088586","seg_speed":50,"src":"0"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X010行驶14.2公里,左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":87,"fb_sign":0,"grade":4,"high_risk":0,"len":0,"light":0,"name":"X010","next_sapa_dist":0,"pointx":"12766046","pointy":"4094255","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"lefty","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":101,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"高阳通达加油站","next_sapa_dist":0,"pointx":"12766913","pointy":"4095415","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"8500961500372617883","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":102,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12766969","pointy":"4095577","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2510573","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":104,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12767041","pointy":"4095762","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"11891987675597381438","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":105,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12767152","pointy":"4095928","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"16301330056795701111","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":110,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12767446","pointy":"4096359","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"8125364181384901895","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":110,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12767461","pointy":"4096345","section_id":0,"section_length":0,"speed":50,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2511102","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":131,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12771670","pointy":"4099059","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"15936391682244171961","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":131,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"688加油站","next_sapa_dist":0,"pointx":"12771816","pointy":"4099171","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"3579147959264360500","url1":"","url2":"","view_url":""}],"tsection":"cross","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":16,"adcode":410221,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12773227.83,4102223.07,-4100,-1412,-1412,4102,4100,1412","coor_start_a":167,"coor_start_b":169,"coor_start_c":175,"pointx_a":"114743670","pointx_b":"114743550","pointx_c":"114738059","pointy_a":"34545073","pointy_b":"34545184","pointy_c":"34545259"}],"connect_len":0,"coorNum":22,"coorStart":146,"directText":"","direction":"","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":6,"inter":[],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[{"coorStart":163,"flag":"bb","lane":"33","pointx":"12773215","pointy":"4101993","type":"00"},{"coorStart":167,"flag":"d00","lane":"104","pointx":"12773206","pointy":"4102228","type":"000"}],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":194,"roadLength":1370,"roadName":"X010","road_names":[{"coorStart":146,"grade":6,"light":4,"name":"X010","pointx":"12774118","pointy":"4100921","section_id":4,"start_distance":0},{"coorStart":148,"grade":6,"light":4,"name":"","pointx":"12774121","pointy":"4100951","section_id":4,"start_distance":0}],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":146,"pointx":"12774118","pointy":"4100921","seg_speed":50,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":6,"textInfo":"沿X010行驶1.4公里,左转","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"进入右转专用道","action":"偏右转","action_length":0,"adcode":410221,"alias":"","br":[{"angle":0,"arrow":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/e0000200","arrow_coors":"","bound_coors":"","confidence":1,"coorStart":176,"coor_start_b":179,"end_pointx":"12771968","end_pointy":"4102271","pattern":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/e0000200","pointx":"12772007","pointy":"4102262","type":20,"uid":443807}],"buildingLength":0,"city_borders":[{"bound_coors":"12772020.21,4102235.38,0,6338,-6339,0,0,-6338","coor_start_a":176,"coor_start_b":179,"coor_start_c":182,"pointx_a":"114732899","pointx_b":"114732549","pointx_c":"114732339","pointy_a":"34545320","pointy_b":"34545389","pointy_c":"34550139"}],"connect_len":0,"coorNum":9,"coorStart":168,"directText":"","direction":"西","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":6,"inter":[{"coorStart":174,"direction":1,"dist":946,"pointx":"12772596","pointy":"4102254","start_distance":946,"tsection":""}],"intersection":21,"intersection_ext":0,"intersection_infos":[],"intersections":[21],"kp":[],"laneinfo":[{"coorStart":174,"flag":"0aa","lane":"D04","pointx":"12772596","pointy":"4102254","type":"000"},{"coorStart":176,"flag":"00b","lane":"104","pointx":"12772007","pointy":"4102262","type":"000"}],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":993,"roadName":"","road_names":[],"roadwidth":9,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":168,"pointx":"12773206","pointy":"4102228","seg_speed":50,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":6,"textInfo":"向西行驶993米,偏右转进入右转专用道","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"直行","action_length":0,"adcode":410221,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12772235.58,4109504.19,2436,84,-85,2437,-2436,-84","coor_start_a":198,"coor_start_b":199,"coor_start_c":213,"pointx_a":"114735049","pointx_b":"114735049","pointx_c":"114734379","pointy_a":"34598979","pointy_b":"34598979","pointy_c":"34603369"}],"connect_len":61,"coorNum":22,"coorStart":177,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":6,"inter":[{"coorStart":195,"direction":1,"dist":370,"pointx":"12772235","pointy":"4109067","start_distance":370,"tsection":""}],"intersection":1,"intersection_ext":0,"intersection_infos":[],"intersections":[66,1],"kp":[],"laneinfo":[{"coorStart":179,"flag":"b","lane":"3","pointx":"12771968","pointy":"4102271","type":"0"},{"coorStart":182,"flag":"aaaa","lane":"0004","pointx":"12771945","pointy":"4102913","type":"0000"},{"coorStart":183,"flag":"0aaa","lane":"1000","pointx":"12771947","pointy":"4103012","type":"0000"},{"coorStart":185,"flag":"0aaa","lane":"1004","pointx":"12771954","pointy":"4103915","type":"0000"},{"coorStart":185,"flag":"0aaa","lane":"1000","pointx":"12771954","pointy":"4104004","type":"0000"},{"coorStart":186,"flag":"0aa0","lane":"1003","pointx":"12771972","pointy":"4106251","type":"0000"},{"coorStart":190,"flag":"0aaa","lane":"1004","pointx":"12771978","pointy":"4106863","type":"0000"}],"light":[{"coorStart":183,"name":"红绿灯","pointx":"12771947","pointy":"4103012","start_distance":0,"tips_type":0},{"coorStart":185,"name":"红绿灯","pointx":"12771954","pointy":"4104004","start_distance":0,"tips_type":0},{"coorStart":186,"name":"红绿灯","pointx":"12771972","pointy":"4106251","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":6019,"roadName":"经四路","road_names":[{"coorStart":177,"grade":6,"light":4,"name":"经四路","pointx":"12772007","pointy":"4102262","section_id":4,"start_distance":0},{"coorStart":186,"grade":6,"light":2,"name":"迎宾大道","pointx":"12771972","pointy":"4106251","section_id":4,"start_distance":116}],"roadwidth":12,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":177,"pointx":"12772007","pointy":"4102262","seg_speed":50,"src":"0"},{"coorStart":181,"pointx":"12771946","pointy":"4102297","seg_speed":60,"src":"0"},{"coorStart":185,"pointx":"12771954","pointy":"4103915","seg_speed":80,"src":"0"},{"coorStart":185,"pointx":"12771955","pointy":"4104103","seg_speed":60,"src":"0"},{"coorStart":194,"pointx":"12772140","pointy":"4108254","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":6,"textInfo":"沿经四路向北行驶6.0公里,直行","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":182,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12771946","pointy":"4102954","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2511092","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":183,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12771947","pointy":"4103039","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2511054","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":183,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12771948","pointy":"4103166","section_id":0,"section_length":0,"speed":60,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2511091","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":183,"fb_sign":0,"grade":6,"high_risk":0,"len":0,"light":1,"name":"经四路","next_sapa_dist":0,"pointx":"12771947","pointy":"4103012","section_id":0,"section_length":8,"speed":0,"start_distance":0,"tips_type":11,"tsection":"cross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"中国石化加油站(杞县妇儿医院南)","next_sapa_dist":0,"pointx":"12772044","pointy":"4104057","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"8748117419200574889","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12771954","pointy":"4103938","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2511135","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12771954","pointy":"4104015","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2511088","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12771957","pointy":"4104427","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2511094","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12771965","pointy":"4105606","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2511131","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12771968","pointy":"4105917","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2511130","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":185,"fb_sign":0,"grade":6,"high_risk":0,"len":0,"light":1,"name":"经四路","next_sapa_dist":0,"pointx":"12771954","pointy":"4104004","section_id":0,"section_length":10,"speed":0,"start_distance":0,"tips_type":11,"tsection":"cross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":186,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违反禁止标线","next_sapa_dist":0,"pointx":"12771971","pointy":"4106216","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":31,"uid":"2511112","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":186,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12771971","pointy":"4106202","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2511111","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":186,"fb_sign":0,"grade":6,"high_risk":0,"len":0,"light":1,"name":"经四路","next_sapa_dist":0,"pointx":"12771972","pointy":"4106251","section_id":0,"section_length":11,"speed":0,"start_distance":0,"tips_type":11,"tsection":"bigcross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":195,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12772238","pointy":"4109094","section_id":0,"section_length":0,"speed":40,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2511837","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":195,"fb_sign":0,"grade":6,"high_risk":0,"len":0,"light":0,"name":"经四路","next_sapa_dist":0,"pointx":"12772235","pointy":"4109067","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"rightic","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":198,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"杞县收费站","next_sapa_dist":0,"pointx":"12772247","pointy":"4109516","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":5,"tsection":"","type":0,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[{"coors":"12772086.86,4104011.80,-8759,334,334,8759,8759,-334","type":6}]},{"accessorialInfo":"","action":"偏右转","action_length":0,"adcode":410221,"alias":"","br":[{"angle":0,"arrow":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/10221009","arrow_coors":"","bound_coors":"","confidence":1,"coorStart":203,"coor_start_b":204,"end_pointx":"12772190","end_pointy":"4109837","pattern":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/10221009","pointx":"12772190","pointy":"4109837","type":14,"uid":598840}],"buildingLength":0,"city_borders":[{"bound_coors":"12772183.00,4109821.82,2304,792,-794,2306,-2304,-792","coor_start_a":203,"coor_start_b":204,"coor_start_c":217,"pointx_a":"114734539","pointx_b":"114734539","pointx_c":"114737580","pointy_a":"34601349","pointy_b":"34601349","pointy_c":"34604739"}],"connect_len":0,"coorNum":5,"coorStart":199,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2080,"funcclass":0,"grade_id":0,"inter":[],"intersection":28,"intersection_ext":0,"intersection_infos":[],"intersections":[28,22],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":269,"roadName":"迎宾大道","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[{"alias":"","name":"连霍高速\/G30","pointx":"12772190","pointy":"4109837","type":1}],"specialGuidance":[],"speed_info":[{"coorStart":199,"pointx":"12772247","pointy":"4109516","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":2080,"sw_grade_id":0,"textInfo":"沿迎宾大道向北行驶269米,偏右转","tips":[],"tsection":"","voice_flag":1,"white_bounds":[]},{"accessorialInfo":"进入匝道","action":"偏右转","action_length":0,"adcode":410221,"alias":"","br":[{"angle":0,"arrow":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/10225121","arrow_coors":"","bound_coors":"","confidence":1,"coorStart":400,"coor_start_b":401,"end_pointx":"12813979","end_pointy":"4108078","pattern":"https:\/\/mvectors.map.qq.com\/drivepic_evonbg\/arrow\/10225121","pointx":"12813979","pointy":"4108078","type":14,"uid":599358}],"buildingLength":0,"city_borders":[{"adcode":411421,"city_name":"商丘市","coor_start":319,"pointx":"12795269.12","pointy":"4110347.44","province_name":""},{"bound_coors":"12813976.54,4108095.16,-1398,-1996,1997,-1398,1398,1996","coor_start_a":400,"coor_start_b":401,"coor_start_c":424,"pointx_a":"115109936","pointx_b":"115109936","pointx_c":"115110429","pointy_a":"34588341","pointy_b":"34588341","pointy_c":"34588109"}],"connect_len":353,"coorNum":197,"coorStart":204,"directText":"","direction":"","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":400,"feat_id":1,"name_info":"","pointx":"115109936","pointy":"34588341"},"fc":0,"fee":2,"form_id":4100,"funcclass":0,"grade_id":0,"inter":[],"intersection":21,"intersection_ext":0,"intersection_infos":[],"intersections":[21],"kp":[],"laneinfo":[{"coorStart":265,"flag":"aa","lane":"04","pointx":"12782557","pointy":"4109817","type":"00"},{"coorStart":266,"flag":"aa","lane":"04","pointx":"12782888","pointy":"4109918","type":"00"},{"coorStart":319,"flag":"aa","lane":"04","pointx":"12795269","pointy":"4110347","type":"00"},{"coorStart":320,"flag":"aa","lane":"04","pointx":"12795563","pointy":"4110368","type":"00"},{"coorStart":372,"flag":"aa","lane":"04","pointx":"12809742","pointy":"4111890","type":"00"},{"coorStart":400,"flag":"000b","lane":"0004","pointx":"12813979","pointy":"4108078","type":"0000"}],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":36549,"roadName":"郑民高速","road_names":[{"coorStart":204,"grade":0,"light":2,"name":"","pointx":"12772190","pointy":"4109837","section_id":2080,"start_distance":0},{"coorStart":216,"grade":0,"light":1,"name":"郑民高速","pointx":"12772277","pointy":"4110209","section_id":4100,"start_distance":0},{"coorStart":372,"grade":0,"light":1,"name":"","pointx":"12809742","pointy":"4111890","section_id":6152,"start_distance":85},{"coorStart":384,"grade":0,"light":1,"name":"连霍高速","pointx":"12810531","pointy":"4111538","section_id":4100,"start_distance":0}],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[{"alias":"","name":"民权\/睢县\/S211","pointx":"12813979","pointy":"4108078","type":2}],"specialGuidance":[],"speed_info":[{"coorStart":204,"pointx":"12772190","pointy":"4109837","seg_speed":40,"src":"1"},{"coorStart":216,"pointx":"12772277","pointy":"4110209","seg_speed":120,"src":"1"},{"coorStart":372,"pointx":"12809742","pointy":"4111890","seg_speed":50,"src":"1"},{"coorStart":384,"pointx":"12810531","pointy":"4111538","seg_speed":120,"src":"1"}],"ss_type":0,"sw_form_id":4100,"sw_grade_id":0,"textInfo":"沿郑民高速行驶36.5公里,偏右转进入匝道","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":266,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"杞县服务区","next_sapa_dist":0,"pointx":"12782888","pointy":"4109918","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":2,"tsection":"","type":0,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":268,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12783238","pointy":"4110010","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2511879","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":268,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12783186","pointy":"4109998","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2511878","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":271,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12783822","pointy":"4110127","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":39,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":317,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12794416","pointy":"4110258","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2511915","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":317,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12794438","pointy":"4110260","section_id":0,"section_length":0,"speed":120,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2511914","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":320,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"保持左侧行驶","next_sapa_dist":0,"pointx":"12795563","pointy":"4110368","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":8,"tsection":"","type":0,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":320,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"郑民高速","next_sapa_dist":0,"pointx":"12795563","pointy":"4110368","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"rightic","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":324,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12796239","pointy":"4110364","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":39,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"进入匝道","active_time":"","coorNum":0,"coorStart":372,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"郑民高速","next_sapa_dist":0,"pointx":"12809742","pointy":"4111890","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"righty","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":386,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12811618","pointy":"4110344","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2517038","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":389,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12812355","pointy":"4109524","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2517086","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":389,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12812386","pointy":"4109489","section_id":0,"section_length":0,"speed":120,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2517087","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":395,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"应急车道摄像头","next_sapa_dist":0,"pointx":"12813305","pointy":"4108590","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":7,"uid":"2517088","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[{"coors":"12782888.04,4109918.85,5644,-25203,93430,20922,-5644,25203","type":1}]},{"accessorialInfo":"","action":"直行","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12813921.33,4108424.57,1894,1534,-1534,1895,-1894,-1534","coor_start_a":433,"coor_start_b":434,"coor_start_c":442,"pointx_a":"115109429","pointx_b":"115109429","pointx_c":"115110099","pointy_a":"34591029","pointy_b":"34591029","pointy_c":"34595059"}],"connect_len":0,"coorNum":33,"coorStart":401,"directText":"","direction":"","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2080,"funcclass":0,"grade_id":0,"inter":[],"intersection":1,"intersection_ext":0,"intersection_infos":[],"intersections":[66,1],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":875,"roadName":"","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"3","co_type":"1","code_source":"183","is_fee_time":0,"name_in":"杞县收费站","name_out":"民权收费站","pay_type":1,"pointx_in":"12772247","pointx_out":"12813923","pointy_in":"4109516","pointy_out":"4108441","price":"20","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":401,"pointx":"12813979","pointy":"4108078","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":2080,"sw_grade_id":0,"textInfo":"行驶875米,直行","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":429,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12814037","pointy":"4108286","section_id":0,"section_length":0,"speed":40,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2517029","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":431,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814013","pointy":"4108325","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":39,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":432,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12813939","pointy":"4108421","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2517069","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":433,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"民权收费站","next_sapa_dist":0,"pointx":"12813923","pointy":"4108441","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":5,"tsection":"","type":0,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12813797.42,4108551.64,1598,1838,-1840,1600,-1598,-1838","coor_start_a":436,"coor_start_b":437,"coor_start_c":443,"pointx_a":"115108289","pointx_b":"115108289","pointx_c":"115111399","pointy_a":"34591969","pointy_b":"34591969","pointy_c":"34596499"}],"connect_len":0,"coorNum":3,"coorStart":434,"directText":"","direction":"西北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2080,"funcclass":0,"grade_id":0,"inter":[],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[{"coorStart":436,"flag":"000bbb","lane":"111333","pointx":"12813796","pointy":"4108568","type":"000000"}],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":148,"roadName":"","road_names":[],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":434,"pointx":"12813923","pointy":"4108441","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":2080,"sw_grade_id":0,"textInfo":"向西北行驶148米,右转","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":17,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12814929.23,4110608.13,-3918,2082,1849,3477,3918,-2082","coor_start_a":453,"coor_start_b":455,"coor_start_c":460,"pointx_a":"115118399","pointx_b":"115118349","pointx_c":"115114500","pointy_a":"34607179","pointy_b":"34607329","pointy_c":"34608031"}],"connect_len":0,"coorNum":17,"coorStart":437,"directText":"","direction":"","driveTime":0,"end_light":1,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":3,"inter":[{"coorStart":446,"direction":3,"dist":1175,"pointx":"12814252","pointy":"4109365","start_distance":1175,"tsection":"cross"},{"coorStart":448,"direction":2,"dist":793,"pointx":"12814459","pointy":"4109780","start_distance":793,"tsection":""}],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[{"coorStart":446,"flag":"aa","lane":"24","pointx":"12814252","pointy":"4109365","type":"00"},{"coorStart":453,"flag":"d0","lane":"24","pointx":"12814921","pointy":"4110625","type":"00"}],"light":[{"coorStart":446,"name":"红绿灯","pointx":"12814252","pointy":"4109365","start_distance":0,"tips_type":0},{"coorStart":453,"name":"红绿灯","pointx":"12814921","pointy":"4110625","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":95,"roadLength":1935,"roadName":"S211","road_names":[],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":437,"pointx":"12813796","pointy":"4108568","seg_speed":70,"src":"1"},{"coorStart":440,"pointx":"12813919","pointy":"4108859","seg_speed":30,"src":"1"},{"coorStart":443,"pointx":"12814146","pointy":"4109188","seg_speed":70,"src":"1"},{"coorStart":451,"pointx":"12814740","pointy":"4110310","seg_speed":30,"src":"1"},{"coorStart":451,"pointx":"12814864","pointy":"4110524","seg_speed":70,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":3,"textInfo":"沿S211行驶1.9公里,左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":441,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12813954","pointy":"4108916","section_id":0,"section_length":0,"speed":30,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2517068","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":441,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12813972","pointy":"4108945","section_id":0,"section_length":0,"speed":30,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2517090","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":442,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814010","pointy":"4109005","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":445,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12814240","pointy":"4109340","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2517030","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":446,"fb_sign":0,"grade":3,"high_risk":0,"len":0,"light":1,"name":"S211","next_sapa_dist":0,"pointx":"12814252","pointy":"4109365","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":11,"tsection":"cross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":450,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"非机动车车道摄像头","next_sapa_dist":0,"pointx":"12814698","pointy":"4110240","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":8,"uid":"2517032","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":451,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814740","pointy":"4110310","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":452,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12814902","pointy":"4110593","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2517045","url1":"","url2":"","view_url":""}],"tsection":"bigcross","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12814499.41,4110661.68,170,2432,-2433,170,-170,-2432","coor_start_a":458,"coor_start_b":459,"coor_start_c":461,"pointx_a":"115114505","pointx_b":"115114505","pointx_c":"115114609","pointy_a":"34607541","pointy_b":"34607541","pointy_c":"34610009"}],"connect_len":0,"coorNum":5,"coorStart":454,"directText":"","direction":"西","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":2,"inter":[],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":370,"roadName":"旺业路","road_names":[],"roadwidth":9,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":454,"pointx":"12814921","pointy":"4110625","seg_speed":60,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":2,"textInfo":"沿旺业路向西行驶370米,右转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":457,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814651","pointy":"4110664","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":2,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"t","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"偏左转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12815465.57,4118064.71,2318,-752,753,2319,-2318,752","coor_start_a":475,"coor_start_b":476,"coor_start_c":479,"pointx_a":"115123423","pointx_b":"115123423","pointx_c":"115124932","pointy_a":"34662220","pointy_b":"34662220","pointy_c":"34666321"}],"connect_len":0,"coorNum":17,"coorStart":459,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":2,"inter":[],"intersection":11,"intersection_ext":0,"intersection_infos":[],"intersections":[11],"kp":[],"laneinfo":[{"coorStart":462,"flag":"a0","lane":"23","pointx":"12814554","pointy":"4112091","type":"00"},{"coorStart":470,"flag":"aa","lane":"24","pointx":"12814932","pointy":"4116350","type":"00"}],"light":[{"coorStart":462,"name":"红绿灯","pointx":"12814554","pointy":"4112091","start_distance":0,"tips_type":0},{"coorStart":470,"name":"红绿灯","pointx":"12814932","pointy":"4116350","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":115,"roadLength":6175,"roadName":"G310","road_names":[{"coorStart":459,"grade":2,"light":2,"name":"G310","pointx":"12814488","pointy":"4110674","section_id":4,"start_distance":0},{"coorStart":470,"grade":3,"light":4,"name":"S324","pointx":"12814932","pointy":"4116350","section_id":2,"start_distance":0},{"coorStart":472,"grade":3,"light":4,"name":"陇海铁路立交桥","pointx":"12815125","pointy":"4116962","section_id":256,"start_distance":0},{"coorStart":472,"grade":3,"light":4,"name":"S324","pointx":"12815306","pointy":"4117536","section_id":2,"start_distance":495}],"roadwidth":9,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":459,"pointx":"12814488","pointy":"4110674","seg_speed":60,"src":"1"},{"coorStart":467,"pointx":"12814634","pointy":"4113894","seg_speed":40,"src":"1"},{"coorStart":471,"pointx":"12814932","pointy":"4116363","seg_speed":50,"src":"1"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":2,"textInfo":"沿G310向北行驶6.2公里,偏左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":460,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814495","pointy":"4110904","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":461,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12814551","pointy":"4112059","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2517076","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":462,"fb_sign":0,"grade":2,"high_risk":0,"len":0,"light":1,"name":"G310","next_sapa_dist":0,"pointx":"12814554","pointy":"4112091","section_id":0,"section_length":13,"speed":0,"start_distance":0,"tips_type":11,"tsection":"","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":467,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12814647","pointy":"4114003","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":31,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":468,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12814849","pointy":"4115675","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2517057","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":468,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12814924","pointy":"4116276","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2517037","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":469,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"路况监控","next_sapa_dist":0,"pointx":"12814929","pointy":"4116322","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":47,"uid":"2517046","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":469,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"闯红灯摄像头","next_sapa_dist":0,"pointx":"12814929","pointy":"4116319","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":1,"uid":"2517047","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":470,"fb_sign":0,"grade":2,"high_risk":0,"len":0,"light":1,"name":"G310","next_sapa_dist":0,"pointx":"12814932","pointy":"4116350","section_id":0,"section_length":10,"speed":0,"start_distance":0,"tips_type":11,"tsection":"bigcross","type":1,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":471,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12814978","pointy":"4116506","section_id":0,"section_length":0,"speed":50,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2517033","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12818326.16,4119235.52,170,-2432,2433,170,-170,2432","coor_start_a":494,"coor_start_b":495,"coor_start_c":501,"pointx_a":"115149099","pointx_b":"115149099","pointx_c":"115149729","pointy_a":"34670730","pointy_b":"34670730","pointy_c":"34675036"}],"connect_len":0,"coorNum":19,"coorStart":476,"directText":"","direction":"","driveTime":0,"end_light":1,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":6,"inter":[],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[{"coorStart":477,"flag":"a","lane":"4","pointx":"12815570","pointy":"4118396","type":"0"},{"coorStart":477,"flag":"a","lane":"0","pointx":"12815577","pointy":"4118418","type":"0"},{"coorStart":494,"flag":"d0","lane":"23","pointx":"12818339","pointy":"4119224","type":"00"}],"light":[{"coorStart":494,"name":"红绿灯","pointx":"12818339","pointy":"4119224","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":2914,"roadName":"西环路","road_names":[{"coorStart":476,"grade":6,"light":5,"name":"西环路","pointx":"12815480","pointy":"4118072","section_id":2,"start_distance":0},{"coorStart":486,"grade":6,"light":5,"name":"北外环路","pointx":"12816018","pointy":"4119090","section_id":2,"start_distance":0}],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":476,"pointx":"12815480","pointy":"4118072","seg_speed":60,"src":"1"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":6,"textInfo":"沿西环路行驶2.9公里,左转","tips":[],"tsection":"cross","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12818328.42,4131157.16,2438,0,0,2439,-2438,0","coor_start_a":545,"coor_start_b":546,"coor_start_c":552,"pointx_a":"115149112","pointx_b":"115149112","pointx_c":"115154689","pointy_a":"34758934","pointy_b":"34758934","pointy_c":"34758109"}],"connect_len":0,"coorNum":51,"coorStart":495,"directText":"","direction":"","driveTime":0,"end_light":1,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[],"light":[{"coorStart":545,"name":"红绿灯","pointx":"12818340","pointy":"4131169","start_distance":0,"tips_type":0}],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":171,"roadLength":9941,"roadName":"程宁线","road_names":[],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":495,"pointx":"12818339","pointy":"4119224","seg_speed":60,"src":"0"},{"coorStart":514,"pointx":"12818559","pointy":"4123499","seg_speed":50,"src":"0"},{"coorStart":517,"pointx":"12818188","pointy":"4124352","seg_speed":60,"src":"0"},{"coorStart":522,"pointx":"12818343","pointy":"4125668","seg_speed":40,"src":"1"},{"coorStart":529,"pointx":"12818391","pointy":"4127039","seg_speed":50,"src":"0"},{"coorStart":532,"pointx":"12818355","pointy":"4128662","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿程宁线行驶9.9公里,右转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":495,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12818415","pointy":"4119393","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2642677","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":495,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"固定测速摄像头","next_sapa_dist":0,"pointx":"12818405","pointy":"4119371","section_id":0,"section_length":0,"speed":60,"start_distance":0,"tips_type":4,"tsection":"","type":3,"uid":"2642678","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":501,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12818365","pointy":"4119867","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"13680734369002418436","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":517,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12818188","pointy":"4124352","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":2,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":523,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12818373","pointy":"4125901","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":527,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"诚信加油站(王庄村)","next_sapa_dist":0,"pointx":"12818427","pointy":"4126677","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"12200104158001094578","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":543,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12818386","pointy":"4130905","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"1247793862769302689","url1":"","url2":"","view_url":""}],"tsection":"cross","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"偏左转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12820440.42,4131017.42,-84,-2436,2437,-85,84,2436","coor_start_a":556,"coor_start_b":557,"coor_start_c":564,"pointx_a":"115168081","pointx_b":"115168081","pointx_c":"115173069","pointy_a":"34757720","pointy_b":"34757720","pointy_c":"34759949"}],"connect_len":0,"coorNum":11,"coorStart":546,"directText":"","direction":"东","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[],"intersection":11,"intersection_ext":0,"intersection_infos":[],"intersections":[11],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":186,"roadLength":1744,"roadName":"X001","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":546,"pointx":"12818340","pointy":"4131169","seg_speed":40,"src":"1"},{"coorStart":546,"pointx":"12818539","pointy":"4131130","seg_speed":30,"src":"1"},{"coorStart":549,"pointx":"12818708","pointy":"4131095","seg_speed":40,"src":"1"},{"coorStart":553,"pointx":"12819737","pointy":"4131025","seg_speed":30,"src":"1"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X001向东行驶1.7公里,偏左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":546,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12818361","pointy":"4131164","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2642823","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":546,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12818539","pointy":"4131130","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":555,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"程东加油站","next_sapa_dist":0,"pointx":"12820407","pointy":"4130984","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"16822077361069531158","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12825448.49,4134874.63,1894,-1534,1534,1895,-1894,1534","coor_start_a":601,"coor_start_b":602,"coor_start_c":604,"pointx_a":"115213118","pointx_b":"115213118","pointx_c":"115217336","pointy_a":"34786288","pointy_b":"34786288","pointy_c":"34785973"}],"connect_len":0,"coorNum":45,"coorStart":557,"directText":"","direction":"","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[{"coorStart":595,"direction":3,"dist":683,"pointx":"12824966","pointy":"4134211","start_distance":683,"tsection":"cross"}],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":74,"roadLength":5463,"roadName":"X001","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":557,"pointx":"12820452","pointy":"4131004","seg_speed":40,"src":"1"},{"coorStart":580,"pointx":"12823162","pointy":"4132128","seg_speed":50,"src":"0"},{"coorStart":582,"pointx":"12823505","pointy":"4132470","seg_speed":40,"src":"3"},{"coorStart":595,"pointx":"12824945","pointy":"4134185","seg_speed":30,"src":"3"},{"coorStart":595,"pointx":"12825094","pointy":"4134388","seg_speed":40,"src":"3"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X001行驶5.5公里,右转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":559,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12820814","pointy":"4131233","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":2,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":583,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12823713","pointy":"4132697","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":32,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":584,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12823878","pointy":"4132890","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":32,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":585,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12823921","pointy":"4132942","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":12,"uid":"0","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":596,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"中石化第十六服务区","next_sapa_dist":0,"pointx":"12825228","pointy":"4134510","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"16561902456877506343","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":598,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12825292","pointy":"4134659","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":32,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[{"coors":"12825174.90,4134499.90,4310,6457,6457,-4310,-4310,-6457","type":6}]},{"accessorialInfo":"","action":"左转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12826699.53,4134768.46,-254,-2424,2425,-254,254,2424","coor_start_a":607,"coor_start_b":608,"coor_start_c":611,"pointx_a":"115224299","pointx_b":"115224299","pointx_c":"115224899","pointy_a":"34785393","pointy_b":"34785393","pointy_c":"34789919"}],"connect_len":0,"coorNum":6,"coorStart":602,"directText":"","direction":"东","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[{"coorStart":603,"direction":1,"dist":696,"pointx":"12825866","pointy":"4134838","start_distance":696,"tsection":""}],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":116,"roadLength":1027,"roadName":"X001","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":602,"pointx":"12825465","pointy":"4134876","seg_speed":40,"src":"1"},{"coorStart":604,"pointx":"12826211","pointy":"4134804","seg_speed":50,"src":"0"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X001向东行驶1.0公里,左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":602,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"加油站","next_sapa_dist":0,"pointx":"12825836","pointy":"4134821","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"16691648661313041472","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":606,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"李堂加油站","next_sapa_dist":0,"pointx":"12826690","pointy":"4134777","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":1,"tsection":"","type":0,"uid":"13139268907205915262","url1":"","url2":"","view_url":""},{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":606,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12826602","pointy":"4134766","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":1,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12826833.32,4136028.24,2434,-126,127,2435,-2434,126","coor_start_a":615,"coor_start_b":616,"coor_start_c":619,"pointx_a":"115225519","pointx_b":"115225519","pointx_c":"115228369","pointy_a":"34794871","pointy_b":"34794871","pointy_c":"34794609"}],"connect_len":0,"coorNum":8,"coorStart":608,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":273,"roadLength":1061,"roadName":"X001","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":608,"pointx":"12826710","pointy":"4134755","seg_speed":50,"src":"0"},{"coorStart":611,"pointx":"12826781","pointy":"4135411","seg_speed":40,"src":"1"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X001向北行驶1.1公里,右转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":614,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"","next_sapa_dist":0,"pointx":"12826841","pointy":"4135979","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":10,"tsection":"","type":2,"uid":"0","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12832644.24,4135317.20,-380,-2406,2408,-381,380,2406","coor_start_a":643,"coor_start_b":644,"coor_start_c":648,"pointx_a":"115277695","pointx_b":"115277695","pointx_c":"115278230","pointy_a":"34789437","pointy_b":"34789437","pointy_c":"34793905"}],"connect_len":0,"coorNum":28,"coorStart":616,"directText":"","direction":"东","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":4,"inter":[],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":4810,"roadName":"X001","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":616,"pointx":"12826846","pointy":"4136039","seg_speed":40,"src":"1"},{"coorStart":619,"pointx":"12827559","pointy":"4135962","seg_speed":50,"src":"0"},{"coorStart":620,"pointx":"12827942","pointy":"4135920","seg_speed":40,"src":"1"},{"coorStart":622,"pointx":"12828714","pointy":"4135836","seg_speed":50,"src":"0"},{"coorStart":632,"pointx":"12830672","pointy":"4135607","seg_speed":40,"src":"1"},{"coorStart":638,"pointx":"12831728","pointy":"4135456","seg_speed":50,"src":"3"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":4,"textInfo":"沿X001向东行驶4.8公里,左转","tips":[{"accessorial_info":"","active_time":"","coorNum":0,"coorStart":626,"fb_sign":0,"grade":0,"high_risk":0,"len":0,"light":0,"name":"违章监控摄像头","next_sapa_dist":0,"pointx":"12829572","pointy":"4135735","section_id":0,"section_length":0,"speed":0,"start_distance":0,"tips_type":4,"tsection":"","type":2,"uid":"2642840","url1":"","url2":"","view_url":""}],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"右转","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12832762.48,4136677.19,2436,-84,85,2437,-2436,84","coor_start_a":656,"coor_start_b":657,"coor_start_c":658,"pointx_a":"115278779","pointx_b":"115278779","pointx_c":"115286953","pointy_a":"34799659","pointy_b":"34799659","pointy_c":"34798721"}],"connect_len":0,"coorNum":13,"coorStart":644,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":6,"inter":[{"coorStart":646,"direction":1,"dist":748,"pointx":"12832708","pointy":"4135781","start_distance":748,"tsection":""},{"coorStart":648,"direction":1,"dist":535,"pointx":"12832733","pointy":"4136039","start_distance":535,"tsection":""}],"intersection":3,"intersection_ext":0,"intersection_infos":[],"intersections":[3],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":1144,"roadName":"","road_names":[],"roadwidth":3,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":644,"pointx":"12832654","pointy":"4135303","seg_speed":50,"src":"0"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":6,"textInfo":"向北行驶1.1公里,右转","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"左转","action_length":10,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[{"bound_coors":"12834694.38,4136412.85,584,3690,3692,-584,-584,-3690","coor_start_a":662,"coor_start_b":664,"coor_start_c":665,"pointx_a":"115296153","pointx_b":"115296272","pointx_c":"115296449","pointy_a":"34797745","pointy_b":"34797730","pointy_c":"34798981"}],"connect_len":0,"coorNum":6,"coorStart":657,"directText":"","direction":"东","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":2,"funcclass":0,"grade_id":6,"inter":[{"coorStart":658,"direction":1,"dist":848,"pointx":"12833684","pointy":"4136561","start_distance":848,"tsection":""},{"coorStart":659,"direction":1,"dist":683,"pointx":"12833884","pointy":"4136542","start_distance":683,"tsection":""},{"coorStart":661,"direction":3,"dist":212,"pointx":"12834453","pointy":"4136468","start_distance":212,"tsection":"cross"}],"intersection":2,"intersection_ext":0,"intersection_infos":[],"intersections":[2],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":212,"roadLength":1602,"roadName":"科技路","road_names":[],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":657,"pointx":"12832774","pointy":"4136688","seg_speed":60,"src":"0"}],"ss_type":0,"sw_form_id":2,"sw_grade_id":6,"textInfo":"沿科技路向东行驶1.6公里,左转","tips":[],"tsection":"t","voice_flag":0,"white_bounds":[]},{"accessorialInfo":"","action":"","action_length":0,"adcode":411421,"alias":"","br":[],"buildingLength":0,"city_borders":[],"connect_len":0,"coorNum":3,"coorStart":663,"directText":"","direction":"北","driveTime":0,"end_light":0,"enter_action":0,"exit_info":{"coor_start":0,"feat_id":0,"name_info":"","pointx":"0","pointy":"0"},"fc":0,"fee":0,"form_id":4,"funcclass":0,"grade_id":3,"inter":[],"intersection":60,"intersection_ext":0,"intersection_infos":[],"intersections":[60],"kp":[],"laneinfo":[],"light":[],"limheight":0,"limspeed":0,"maxlanes":0,"minlanes":0,"parks":[],"prev_inter_dist":0,"roadLength":151,"roadName":"S211","road_names":[],"roadwidth":6,"roundabout":{"exits":[],"flip":0,"type":0},"seg_hints":[],"segmenttoll":{"co_source":"","co_type":"","code_source":"","is_fee_time":1,"name_in":"","name_out":"","pay_type":0,"pointx_in":"0","pointx_out":"0","pointy_in":"0","pointy_out":"0","price":"","type":0},"sound_id":0,"sound_ids":[],"sp":[],"specialGuidance":[],"speed_info":[{"coorStart":663,"pointx":"12834709","pointy":"4136429","seg_speed":70,"src":"0"}],"ss_type":0,"sw_form_id":4,"sw_grade_id":3,"textInfo":"沿S211向北行驶151米,","tips":[],"tsection":"","voice_flag":0,"white_bounds":[]}],"srouteid":"7082286592709298092","startInfo":{"dir":"","distance":0,"sText":""},"start_adsorb_type":2,"statistics":"","tag":"推荐方案","tagv2":"推荐方案","taxi":{"dist":0,"fee":0,"fees":[],"night_time":""},"time":133,"time_s":7980,"toll_txt":"","traffic":[{"c":0,"d":59,"f":0,"roads":[{"d":59,"e":128,"n":"S325"}],"s":29,"t":1},{"c":1,"d":178,"f":1,"roads":[{"d":178,"e":60,"n":"S325"}],"s":19,"t":4},{"c":0,"d":7824,"f":4,"roads":[{"d":7824,"e":571,"n":"S325"}],"s":47,"t":36},{"c":0,"d":14168,"f":36,"roads":[{"d":14168,"e":1165,"n":"X010"}],"s":41,"t":145},{"c":0,"d":1370,"f":145,"roads":[{"d":15,"e":1,"n":"X010"},{"d":1355,"e":121,"n":""}],"s":45,"t":167},{"c":0,"d":993,"f":167,"roads":[{"d":993,"e":75,"n":""}],"s":53,"t":176},{"c":0,"d":977,"f":176,"roads":[{"d":977,"e":90,"n":"经四路"}],"s":46,"t":184},{"c":1,"d":416,"f":184,"roads":[{"d":416,"e":37,"n":"经四路"}],"s":47,"t":185},{"c":0,"d":4254,"f":185,"roads":[{"d":1923,"e":143,"n":"经四路"},{"d":11,"e":0,"n":""},{"d":2319,"e":140,"n":"迎宾大道"}],"s":61,"t":195},{"c":2,"d":259,"f":195,"roads":[{"d":259,"e":14,"n":"迎宾大道"}],"s":79,"t":197},{"c":0,"d":111,"f":197,"roads":[{"d":111,"e":39,"n":"迎宾大道"}],"s":12,"t":198},{"c":0,"d":269,"f":198,"roads":[{"d":269,"e":29,"n":"迎宾大道"}],"s":39,"t":203},{"c":0,"d":36549,"f":203,"roads":[{"d":353,"e":37,"n":""},{"d":31408,"e":1525,"n":"郑民高速"},{"d":755,"e":49,"n":""},{"d":4031,"e":202,"n":"连霍高速"}],"s":81,"t":400},{"c":0,"d":875,"f":400,"roads":[{"d":875,"e":119,"n":""}],"s":42,"t":433},{"c":0,"d":148,"f":433,"roads":[{"d":148,"e":29,"n":""}],"s":21,"t":436},{"c":0,"d":1935,"f":436,"roads":[{"d":1935,"e":214,"n":"S211"}],"s":35,"t":453},{"c":0,"d":386,"f":453,"roads":[{"d":18,"e":4,"n":""},{"d":367,"e":51,"n":"旺业路"}],"s":30,"t":458},{"c":0,"d":4644,"f":458,"roads":[{"d":4644,"e":376,"n":"G310"}],"s":44,"t":469},{"c":4,"d":43,"f":469,"roads":[{"d":43,"e":7,"n":"G310"}],"s":24,"t":470},{"c":0,"d":1486,"f":470,"roads":[{"d":10,"e":1,"n":""},{"d":517,"e":76,"n":"S324"},{"d":495,"e":72,"n":"陇海铁路立交桥"},{"d":463,"e":50,"n":"S324"}],"s":30,"t":475},{"c":0,"d":2914,"f":475,"roads":[{"d":1002,"e":74,"n":"西环路"},{"d":1911,"e":134,"n":"北外环路"}],"s":55,"t":494},{"c":0,"d":9941,"f":494,"roads":[{"d":9941,"e":730,"n":"程宁线"}],"s":41,"t":545},{"c":0,"d":1744,"f":545,"roads":[{"d":1744,"e":178,"n":"X001"}],"s":39,"t":556},{"c":0,"d":5463,"f":556,"roads":[{"d":5463,"e":462,"n":"X001"}],"s":43,"t":601},{"c":0,"d":1027,"f":601,"roads":[{"d":1027,"e":106,"n":"X001"}],"s":39,"t":607},{"c":0,"d":1061,"f":607,"roads":[{"d":1061,"e":102,"n":"X001"}],"s":42,"t":615},{"c":0,"d":4810,"f":615,"roads":[{"d":4810,"e":421,"n":"X001"}],"s":43,"t":643},{"c":0,"d":1144,"f":643,"roads":[{"d":1144,"e":95,"n":""}],"s":50,"t":656},{"c":0,"d":1602,"f":656,"roads":[{"d":1602,"e":119,"n":"科技路"}],"s":57,"t":662},{"c":0,"d":151,"f":662,"roads":[{"d":10,"e":0,"n":"科技路"},{"d":140,"e":129,"n":"S211"}],"s":60,"t":665}],"traffic_overview":0,"traffictime":133,"traffictime_s":7975}]},"info":{"cache":"false","count":23,"cross_city":0,"dest":{"addr":"","ccode":1667,"cname":"","isxp":0,"name":"","phone":"","pointx":"12834774","pointy":"4136614","poitype":0,"query":"","uid":"","xpinfo":{}},"distance":106788,"error":0,"error_type":"","fork_pts":[],"hassub":0,"message":"ok","nav_session_id":"6631727627328652970_335955450512619630","navtraffic":0,"pass":[],"start":{"addr":"","ccode":1548,"cname":"","isxp":0,"name":"","phone":"","pointx":"12754110","pointy":"4086134","poitype":0,"query":"","uid":"","xpinfo":{}},"start_hint_describe":"","start_hint_type":0,"taxi":{"dist":0,"fee":0,"fees":[],"night_time":""},"time":132,"toolong":0,"traffictag":2,"type":44},"limit_status":{"display_text":"开封市有车牌限行，点击设置，避让限行路段","limited_cities":["开封市"],"nav_route_limit_status":0,"status":1},"perf_info":{"adsorb_time":13,"direct_distance":78315,"guide_time":15,"imei":"866564045026273","logid":2073472820,"queue_time":0,"rank_time":128,"route_distance":0,"search_map":{},"search_time":56,"total_time":223,"traceid":-921823788}}
"""

implicit val formats = DefaultFormats + FieldSerializer[Response]()

val obj = read[Response](v)
println(obj)
